# Compression Pipeline:


<div style="background-color: white; padding: 10px;">
    <img src="./docs/static/img/pipeline.svg" alt="SVG Image" width="1500px" />
</div>

Instead of running the compression pipeline all at once, here you can run it step-by-step and explore the process.

## First simulate the command-line arguments

In [ ]:
simulated_args = [
        "--model_path", "./input_models/flower_hq",
        "--data_device", "cuda",
        "--output_vq", "./output"
]

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import time

from argparse import ArgumentParser
from arguments import (
    CompressionParams,
    ModelParams,
    OptimizationParams,
    PipelineParams,
    get_combined_args,
)

from compress import unique_output_folder, calc_importance
from gaussian_renderer import GaussianModel
from scene import Scene
from compression.vq import CompressionSettings
from typing import Tuple

In [ ]:
def parse_arguments(simulated_args=[]):
    # Initialize the argument parser
    parser = ArgumentParser(description="Compression script parameters")
    
    # Add the same argument groups as in the script
    model = ModelParams(parser, sentinel=True)
    model.data_device = "cuda"
    pipeline = PipelineParams(parser)
    op = OptimizationParams(parser)
    comp = CompressionParams(parser)
    
    # Combine simulated args with parser arguments
    args = get_combined_args(parser, simulated_args)
    return args, model, pipeline, op, comp


args, model, pipeline, op, comp = parse_arguments(simulated_args)

# Set output folder if not specified
if args.output_vq is None:
    args.output_vq = unique_output_folder()

# Extract parameters
model_params = model.extract(args)
optim_params = op.extract(args)
pipeline_params = pipeline.extract(args)
comp_params = comp.extract(args)

In [ ]:
# Initialize the Gaussians
gaussians = GaussianModel(
    model_params.sh_degree, quantization=not optim_params.not_quantization_aware
)

# Initialize the scene (test cameras + train cameras)
scene = Scene(
    model_params, gaussians, load_iteration=comp_params.load_iteration, shuffle=True
)

# Load the Gaussians from the pre-trained model (checkpoint) into memory
if comp_params.start_checkpoint:
    (checkpoint_params, first_iter) = torch.load(comp_params.start_checkpoint)
    gaussians.restore(checkpoint_params, optim_params)


timings = {}

## Step 1: Parameter Sensitivity
Note: The authors use 'sensitivity' and 'importance' interchangeably, this is very confusing I know 

In [ ]:
start_time = time.time()

color_importance, gaussian_sensitivity = calc_importance(
    gaussians, scene, pipeline_params
)
end_time = time.time()
timings["sensitivity_calculation"] = end_time-start_time

In [ ]:
color_importance_include = torch.tensor(0.6 * 1e-6)
gaussian_importance_include = torch.tensor(0.3 * 1e-5)

color_above_threshold = (color_importance > color_importance_include).sum().item()
total_elements_color = color_importance.numel()

gaussian_above_threshold = (gaussian_sensitivity > gaussian_importance_include).sum().item()
total_elements_gaussian = gaussian_sensitivity.numel()

color_threshold = 1.0 - (color_above_threshold / total_elements_color)
gaussian_threshold = 1.0 - (gaussian_above_threshold / total_elements_gaussian)

print(f"Percentage of color_importance values below the threshold: {color_threshold:.2f}%")
print(f"Percentage of gaussian_importance values below the threshold: {gaussian_threshold:.2f}%")

In [ ]:
# Normalize the tensors
color_importance_norm = torch.nn.functional.normalize(color_importance.clone(), p=2)
gaussian_sensitivity_norm = torch.nn.functional.normalize(gaussian_sensitivity.clone(), p=2)

# # Normalize the tensors
# color_importance_norm = color_importance / color_importance.max()
# gaussian_sensitivity_norm = gaussian_sensitivity / gaussian_sensitivity.max()

In [ ]:
color_importance_norm_np = color_importance_norm.cpu().numpy().flatten()  # Convert to numpy array if needed
gaussian_sensitivity_norm_np = gaussian_sensitivity_norm.cpu().numpy().flatten()
# color_threshold = color_importance_include.cpu().numpy()
# gaussian_threshold = gaussian_importance_include.cpu().numpy()

# color_importance = color_importance.flatten()  # Convert to numpy array if needed
# gaussian_sensitivity = gaussian_sensitivity.flatten()


# Define the number of bins
num_bins = 20

# Plotting
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# Color sensitivity histogram
axes[0].hist(
    color_importance_norm_np,
    bins=num_bins,
    color="#1f77b4",
    density=True
)
axes[0].axvline(color_threshold, color='red', linestyle='--', label=f'Threshold ({color_threshold})')
axes[0].set_title("Color Sensitivity Distribution")
axes[0].set_xlabel("Sensitivity")
axes[0].set_ylabel("Density")
# axes[0].set_yscale("log")

# Shape sensitivity histogram
axes[1].hist(
    gaussian_sensitivity_norm_np,
    bins=num_bins,
    color="#ff7f0e",
    density=True
)
axes[1].axvline(gaussian_threshold, color='red', linestyle='--', label=f'Threshold ({gaussian_threshold})')
axes[1].set_title("Shape Sensitivity Distribution")
axes[1].set_xlabel("Sensitivity")

plt.tight_layout()
plt.show()

## Step 2: Sensitivity-aware vector clustering
Note: vector clustering = vector quantization = K-Means clustering

In [ ]:
def prune_gaussians(
    gaussians: GaussianModel,
    color_importance_n: torch.Tensor,
    gaussian_importance_n: torch.Tensor,
    prune_threshold:float=0.,
):
    with torch.no_grad():
        if prune_threshold >= 0:
            non_prune_mask = color_importance_n > prune_threshold
            print(f"prune: {(1-non_prune_mask.float().mean())*100:.2f}%")
            gaussians.mask_splats(non_prune_mask)
            gaussian_importance_n = gaussian_importance_n[non_prune_mask]
            color_importance_n = color_importance_n[non_prune_mask]
        
        # if prune_threshold >= 0:
        #     non_prune_mask = color_importance_n > prune_threshold
        #     prune_percentage = (1 - non_prune_mask.float().mean()) * 100

        #     gaussians.mask_splats(non_prune_mask)
        #     gaussian_importance_n = gaussian_importance_n[non_prune_mask]
        #     color_importance_n = color_importance_n[non_prune_mask]

            # # Get positions for each Gaussian
            # positions = gaussians.get_xyz
            
            # # Separate the positions and sensitivities based on pruning mask
            # pos_keep = positions[non_prune_mask].cpu().numpy()
            # pos_prune = positions[~non_prune_mask].cpu().numpy()

            # fig = go.Figure()

            # # Scatter plot for Gaussians that will be kept
            # fig.add_trace(go.Scatter3d(
            #     x=pos_prune[:, 0],
            #     y=pos_prune[:, 1],
            #     z=pos_prune[:, 2],
            #     mode='markers',
            #     marker=dict(
            #         size=3,
            #         color='red',    
            #         opacity=0.5,
            #     ),
            #     name="Pruned Gaussians"
            # ))

            # # Scatter plot for Gaussians that will be pruned
            # fig.add_trace(go.Scatter3d(
            #     x=pos_keep[:, 0],
            #     y=pos_keep[:, 1],
            #     z=pos_keep[:, 2],
            #     mode='markers',
            #     marker=dict(
            #         size=3,
            #         color='blue',
            #         opacity=0.7,
            #     ),
            #     name="Kept Gaussians"
            # ))

            # title = f"Pruning {prune_percentage:.2f}% of Gaussians"
            # fig.update_layout(
            #     title=title,
            #     scene=dict(
            #         xaxis_title="X",
            #         yaxis_title="Y",
            #         zaxis_title="Z",
            #     ),
            #     legend=dict(x=1, y=0.9)
            # )

            # fig.show()


# TODO: Use point cloud RGB color instead of 'blue'

# from plyfile import PlyData
# import numpy as np
# import plotly.graph_objects as go
# import matplotlib.colors as mcolors

# # Load points3d.ply
# ply_path = "./datasets/flowers/sparse/0/points3D.ply"
# plydata = PlyData.read(ply_path)

# # Extract positions and colors
# # positions_ply = np.vstack([plydata['vertex']['x'], plydata['vertex']['y'], plydata['vertex']['z']]).T
# colors_ply = np.vstack([plydata['vertex']['red'], plydata['vertex']['green'], plydata['vertex']['blue']]).T

# def prune_gaussians(
#     gaussians: GaussianModel,
#     color_importance: torch.Tensor,
#     gaussian_importance: torch.Tensor,
#     prune_threshold: float = 0.,
# ):
#     with torch.no_grad():
#         if prune_threshold >= 0:
#             non_prune_mask = color_importance > prune_threshold
#             prune_percentage = (1 - non_prune_mask.float().mean()) * 100

#             # Get positions and colors for each Gaussian
#             positions = gaussians.get_xyz
#             # colors = gaussians._features_dc  # Assuming _features_dc holds the RGB color

#             # Separate the positions and colors based on pruning mask
#             pos_keep = positions[non_prune_mask].cpu().numpy()
#             pos_prune = positions[~non_prune_mask].cpu().numpy()
#             color_keep = colors_ply[non_prune_mask.cpu().numpy()]  # Colors for kept Gaussians

#             # Normalize colors to range [0, 1] if necessary
#             if color_keep.max() > 1:
#                 color_keep = color_keep / 255.0

#             color_keep = np.clip(color_keep, 0, 1)

#             # Convert RGB colors to hex format for plotly
#             color_keep_hex = [mcolors.to_hex(c) for c in color_keep]

#             fig = go.Figure()

#             # Scatter plot for Gaussians that will be pruned
#             fig.add_trace(go.Scatter3d(
#                 x=pos_prune[:, 0],
#                 y=pos_prune[:, 1],
#                 z=pos_prune[:, 2],
#                 mode='markers',
#                 marker=dict(
#                     size=3,
#                     color='red',    
#                     opacity=0.5,
#                 ),
#                 name="Pruned Gaussians"
#             ))

#             # Scatter plot for Gaussians that will be kept, using actual colors
#             fig.add_trace(go.Scatter3d(
#                 x=pos_keep[:, 0],
#                 y=pos_keep[:, 1],
#                 z=pos_keep[:, 2],
#                 mode='markers',
#                 marker=dict(
#                     size=3,
#                     color=color_keep_hex,  # Use hex colors for kept Gaussians
#                     opacity=0.7,
#                 ),
#                 name="Kept Gaussians"
#             ))

#             title = f"Pruning {prune_percentage:.2f}% of Gaussians"
#             fig.update_layout(
#                 title=title,
#                 scene=dict(
#                     xaxis_title="X",
#                     yaxis_title="Y",
#                     zaxis_title="Z",
#                 ),
#                 legend=dict(x=1, y=0.9)
#             )

#             fig.show()

# import plotly.graph_objects as go
# import matplotlib.colors as mcolors

# def prune_gaussians(
#     gaussians: GaussianModel,
#     color_importance: torch.Tensor,
#     gaussian_importance: torch.Tensor,
#     prune_threshold: float = 0.,
# ):
#     with torch.no_grad():
#         if prune_threshold >= 0:
#             non_prune_mask = color_importance > prune_threshold
#             prune_percentage = (1 - non_prune_mask.float().mean()) * 100

#             # Get positions and colors for each Gaussian
#             positions = gaussians.get_xyz
#             colors = gaussians._features_dc  # Assuming _features_dc holds the RGB color

#             # Separate the positions and colors based on pruning mask
#             pos_keep = positions[non_prune_mask].cpu().numpy()
#             pos_prune = positions[~non_prune_mask].cpu().numpy()
#             color_keep = colors[non_prune_mask].cpu().numpy()  # Colors for kept Gaussians

#             # Ensure color_keep values are within the [0, 1] range
#             color_keep = np.clip(color_keep, 0, 1)  # Clip any values outside [0, 1]

#             # Convert RGB colors to hex format for plotly
#             color_keep_hex = [mcolors.to_hex(c) for c in color_keep]

#             fig = go.Figure()

#             # Scatter plot for Gaussians that will be pruned
#             fig.add_trace(go.Scatter3d(
#                 x=pos_prune[:, 0],
#                 y=pos_prune[:, 1],
#                 z=pos_prune[:, 2],
#                 mode='markers',
#                 marker=dict(
#                     size=3,
#                     color='red',    
#                     opacity=0.5,
#                 ),
#                 name="Pruned Gaussians"
#             ))

#             # Scatter plot for Gaussians that will be kept, using actual colors
#             fig.add_trace(go.Scatter3d(
#                 x=pos_keep[:, 0],
#                 y=pos_keep[:, 1],
#                 z=pos_keep[:, 2],
#                 mode='markers',
#                 marker=dict(
#                     size=3,
#                     color=color_keep_hex,  # Use hex colors for kept Gaussians
#                     opacity=0.7,
#                 ),
#                 name="Kept Gaussians"
#             ))

#             title = f"Pruning {prune_percentage:.2f}% of Gaussians"
#             fig.update_layout(
#                 title=title,
#                 scene=dict(
#                     xaxis_title="X",
#                     yaxis_title="Y",
#                     zaxis_title="Z",
#                 ),
#                 legend=dict(x=1, y=0.9)
#             )

#             fig.show()

#             # Actually prune
#             # gaussians.mask_splats(non_prune_mask)
#             # gaussian_importance = gaussian_importance[non_prune_mask]
#             # color_importance = color_importance[non_prune_mask]


------------------ Pruning ------------------

In [ ]:
# Important hyperparameters
print('prune_threshold:', comp_params.prune_threshold)

In [ ]:
with torch.no_grad():
    start_time = time.time()

    # Given a vector x ∈ R^D, we define its sensitivity as the maximum over its component’s sensitivity
    color_importance_n = color_importance.amax(-1)
    gaussian_importance_n = gaussian_sensitivity.amax(-1)

    torch.cuda.empty_cache()

    # ------------------ Prune ------------------
    prune_threshold = comp_params.prune_threshold

    if prune_threshold >= 0:
        non_prune_mask = color_importance_n > prune_threshold

        # # Get positions for each Gaussian
        # positions = gaussians.get_xyz
        
        # # Separate the positions and sensitivities based on pruning mask
        # pos_keep = positions[non_prune_mask].cpu().numpy()
        # pos_prune = positions[~non_prune_mask].cpu().numpy()

        # fig = go.Figure()

        # # Scatter plot for Gaussians that will be kept
        # fig.add_trace(go.Scatter3d(
        #     x=pos_prune[:, 0],
        #     y=pos_prune[:, 1],
        #     z=pos_prune[:, 2],
        #     mode='markers',
        #     marker=dict(
        #         size=3,
        #         color='red',    
        #         opacity=0.5,
        #     ),
        #     name="Pruned Gaussians"
        # ))

        # # Scatter plot for Gaussians that will be pruned
        # fig.add_trace(go.Scatter3d(
        #     x=pos_keep[:, 0],
        #     y=pos_keep[:, 1],
        #     z=pos_keep[:, 2],
        #     mode='markers',
        #     marker=dict(
        #         size=3,
        #         color='blue',
        #         opacity=0.7,
        #     ),
        #     name="Kept Gaussians"
        # ))

        # prune_percentage = (1 - non_prune_mask.float().mean()) * 100

        # title = f"Pruning {prune_percentage:.2f}% of Gaussians"
        # fig.update_layout(
        #     title=title,
        #     scene=dict(
        #         xaxis_title="X",
        #         yaxis_title="Y",
        #         zaxis_title="Z",
        #     ),
        #     legend=dict(x=1, y=0.9)
        # )

        # fig.show()

        gaussians.mask_splats(non_prune_mask)
        gaussian_importance_n = gaussian_importance_n[non_prune_mask]
        color_importance_n = color_importance_n[non_prune_mask]

------------------ Color Compression ------------------

In [ ]:
# Important hyperparameters
comp_params.color_codebook_size             # K = number of centroids = codebook size
comp_params.color_importance_prune
comp_params.color_importance_include
comp_params.color_cluster_iterations = 1 # TODO: remove
comp_params.color_decay
comp_params.color_batch_size

# Initialize the color codebook using parameters
color_compression_settings = CompressionSettings(
    codebook_size=comp_params.color_codebook_size,
    importance_prune=comp_params.color_importance_prune,
    importance_include=comp_params.color_importance_include,
    steps=int(comp_params.color_cluster_iterations),
    decay=comp_params.color_decay,
    batch_size=comp_params.color_batch_size,
)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# def visualise_feature_space(features: torch.Tensor, use_tsne=False, title="2D Projection of Color Features"):
#     features_np = features.cpu().numpy()

#     if use_tsne: # Careful: takes roughly 15 minutes to run (on my GPU)
#         tsne = TSNE(n_components=2, random_state=0, perplexity=30, n_iter=300)
#         features_2d = tsne.fit_transform(features_np)
#     else:
#         pca = PCA(n_components=2)
#         features_2d = pca.fit_transform(features_np)

#     plt.figure(figsize=(10, 8))
#     plt.scatter(features_2d[:, 0], features_2d[:, 1], s=1, alpha=0.5)
#     plt.title(title)
#     plt.xlabel("Component 1")
#     plt.ylabel("Component 2")
#     plt.show()

# def visualise_feature_space(features: torch.Tensor, assignments: torch.Tensor, use_tsne=False, title="2D Projection of Color Features"):
#     features_np = features.cpu().numpy()
#     assignments_np = assignments.cpu().numpy()  # Convert assignments to numpy

#     # Perform dimensionality reduction
#     if use_tsne:  # t-SNE is computationally expensive
#         tsne = TSNE(n_components=2, random_state=0, perplexity=30, n_iter=300)
#         features_2d = tsne.fit_transform(features_np)
#     else:
#         pca = PCA(n_components=2)
#         features_2d = pca.fit_transform(features_np)

#     # Get unique centroids and assign a color to each
#     unique_assignments = np.unique(assignments_np)
#     colors = plt.cm.get_cmap('tab10', len(unique_assignments))

#     plt.figure(figsize=(10, 8))

#     # Plot each feature colored by its assigned centroid
#     for i, centroid_idx in enumerate(unique_assignments):
#         centroid_features = features_2d[assignments_np == centroid_idx]
#         plt.scatter(centroid_features[:, 0], centroid_features[:, 1], s=1, color=colors(i), alpha=0.3, label=f"Centroid {centroid_idx}")

#     plt.title(title)
#     plt.xlabel("Component 1")
#     plt.ylabel("Component 2")
#     plt.legend()
#     plt.show()

def visualise_feature_space_centroids(features: torch.Tensor, centroids: torch.Tensor, use_tsne=False, title="2D Projection of Color Features"):
    features_np = features.cpu().numpy()
    centroids_np = centroids.cpu().numpy()

    if use_tsne: # Careful: takes roughly 15 minutes to run (on my GPU)
        tsne = TSNE(n_components=2, random_state=0, perplexity=30, n_iter=300)
        features_2d = tsne.fit_transform(features_np)
        centroids_2d = tsne.fit_transform(centroids_np)
    else:
        pca = PCA(n_components=2)
        features_2d = pca.fit_transform(features_np)
        centroids_2d = pca.fit_transform(centroids_np)

    plt.figure(figsize=(10, 8))

    plt.scatter(features_2d[:, 0], features_2d[:, 1], s=1, alpha=0.5)
    plt.scatter(centroids_2d[:, 0], centroids_2d[:, 1], color='red', s=1, label="Centroids")
    
    plt.title(title)
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    plt.show()

In [ ]:
from compression.vq import VectorQuantize
from tqdm import trange
import gc

def vq_features_vis(
    features: torch.Tensor,
    importance: torch.Tensor,
    codebook_size: int,
    vq_chunk: int = 2**16,
    steps: int = 1000,
    decay: float = 0.8,
    scale_normalize: bool = False,
) -> Tuple[torch.Tensor, torch.Tensor]:
    importance_n = importance/importance.max()
    vq_model = VectorQuantize(
        channels=features.shape[-1],
        codebook_size=codebook_size,
        decay=decay,
    ).to(device=features.device)

    vq_model.uniform_init(features)

    errors = []

    for i in trange(steps):
        batch = torch.randint(low=0, high=features.shape[0], size=[vq_chunk])
        vq_feature = features[batch]
        error = vq_model.update(vq_feature, importance=importance_n[batch]).mean().item()
        errors.append(error)

        _, idx = vq_model(features)
        # visualise_feature_space(features, idx, use_tsne=False, title=f"2D Projection of Color Features - Iteration {i+1}")
        visualise_feature_space_centroids(features, vq_model.codebook.data, use_tsne=False, title=f"2D Projection of Color Features - Iteration {i+1}")

        if scale_normalize:
            # this computes the trace of the codebook covariance matrices
            # we devide by the trace to ensure that matrices have normalized eigenvalues / scales
            tr = vq_model.codebook[:, [0, 3, 5]].sum(-1)
            vq_model.codebook /= tr[:, None]

    gc.collect()
    torch.cuda.empty_cache()

    start = time.time()
    _, vq_indices = vq_model(features)
    torch.cuda.synchronize(device=vq_indices.device)
    end = time.time()
    print(f"calculating indices took {end-start} seconds ")
    return vq_model.codebook.data.detach(), vq_indices.detach(), errors

In [ ]:
from compression.vq import join_features

def compress_color_vis(
    gaussians: GaussianModel,
    color_importance_n: torch.Tensor,
    color_comp: CompressionSettings,
    color_compress_non_dir: bool,
):
    keep_mask = color_importance_n > color_comp.importance_include

    print(f"color keep: {keep_mask.float().mean()*100:.2f}%")

    vq_mask_c = ~keep_mask

    # remove zero sh component
    if color_compress_non_dir:
        n_sh_coefs = gaussians.get_features.shape[1]
        color_features = gaussians.get_features.detach().flatten(-2)
    else:
        n_sh_coefs = gaussians.get_features.shape[1] - 1
        color_features = gaussians.get_features[:, 1:].detach().flatten(-2)

    if vq_mask_c.any():
        color_codebook, color_vq_indices, errors = vq_features_vis(
            color_features[vq_mask_c],
            color_importance_n[vq_mask_c],
            color_comp.codebook_size,
            color_comp.batch_size,
            color_comp.steps,
        )
    else:
        color_codebook = torch.empty(
            (0, color_features.shape[-1]), device=color_features.device
        )
        color_vq_indices = torch.empty(
            (0,), device=color_features.device, dtype=torch.long
        )

    all_features = color_features
    compressed_features, indices = join_features(
        all_features, keep_mask, color_codebook, color_vq_indices
    )

    gaussians.set_color_indexed(compressed_features.reshape(-1, n_sh_coefs, 3), indices)

    return errors

In [ ]:
color_errors = []

with torch.no_grad():
    color_comp = color_compression_settings if not comp_params.not_compress_color else None
    if color_comp is not None:
        color_errors = compress_color_vis(
            gaussians,
            color_importance_n,
            color_comp,
            comp_params.color_compress_non_dir,
        )

In [ ]:
def plot_error_curve(errors=[]):
    plt.figure(figsize=(10, 6))
    plt.plot(errors, label='Quantization Error')
    plt.xlabel('Iteration')
    plt.ylabel('Quantization Error')
    plt.title('Quantization Error Over Iterations')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_error_curve(color_errors)

------------------ Gaussian Shape Compression ------------------

In [ ]:
# Important hyperparameters:
comp_params.gaussian_codebook_size
comp_params.gaussian_importance_include
comp_params.gaussian_cluster_iterations = 10 # TODO: remove
comp_params.gaussian_decay
comp_params.gaussian_batch_size

# Initialize the Gaussian shape codebook using parameters
gaussian_compression_settings = CompressionSettings(
    codebook_size=comp_params.gaussian_codebook_size,
    importance_prune=None,
    importance_include=comp_params.gaussian_importance_include,
    steps=int(comp_params.gaussian_cluster_iterations),
    decay=comp_params.gaussian_decay,
    batch_size=comp_params.gaussian_batch_size,
)

In [ ]:
from utils.splats import to_full_cov, extract_rot_scale

def compress_covariance_vis(
    gaussians: GaussianModel,
    gaussian_importance_n: torch.Tensor,
    gaussian_comp: CompressionSettings,
):

    keep_mask_g = gaussian_importance_n > gaussian_comp.importance_include

    vq_mask_g = ~keep_mask_g

    print(f"gaussians keep: {keep_mask_g.float().mean()*100:.2f}%")

    covariance = gaussians.get_normalized_covariance(strip_sym=True).detach()

    if vq_mask_g.any():
        cov_codebook, cov_vq_indices, errors = vq_features_vis(
            covariance[vq_mask_g],
            gaussian_importance_n[vq_mask_g],
            gaussian_comp.codebook_size,
            gaussian_comp.batch_size,
            gaussian_comp.steps,
            scale_normalize=True,
        )
    else:
        cov_codebook = torch.empty(
            (0, covariance.shape[1], 1), device=covariance.device
        )
        cov_vq_indices = torch.empty((0,), device=covariance.device, dtype=torch.long)

    compressed_cov, cov_indices = join_features(
        covariance,
        keep_mask_g,
        cov_codebook,
        cov_vq_indices,
    )

    rot_vq, scale_vq = extract_rot_scale(to_full_cov(compressed_cov))

    gaussians.set_gaussian_indexed(
        rot_vq.to(compressed_cov.device),
        scale_vq.to(compressed_cov.device),
        cov_indices,
    )

    return errors

In [ ]:
shape_errors = []

with torch.no_grad():
    gaussian_comp = gaussian_compression_settings if not comp_params.not_compress_gaussians else None
    if gaussian_comp is not None:
        shape_errors = compress_covariance_vis(
            gaussians,
            gaussian_importance_n,
            gaussian_comp,
        )

end_time = time.time()
timings["clustering"]=end_time-start_time

In [ ]:
plot_error_curve(shape_errors)